# Fusion

Submission 2:
    - Tremor : Gradient Boosting Regression
    - Dyskinesia : Gradient Boosting Regression
    
Submission 4: 
    - Dyskinesia : Average of predictions 

In [87]:
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
import csv
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from random import random
from sklearn.metrics import mean_squared_error
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

%load_ext autoreload
%autoreload 2

from transform_data import *
from create_graphs import *
from beatPDivec.default_data.v2_auto.local.get_final_scores_accuracy import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
sDirOut = "/export/b19/mpgill/BeatPD_predictions_tryingout/"

## Submission 4 - Average of predictions 

In [89]:
def average_fusion(lFilesPred, sDirOut, fileName, bRound, sFileLabels, sTypeLabel, bTestSubset=False):
    
    # Read file labels (true labels)
    if not bTestSubset:
        print("sFileLabels : ", sFileLabels)
        with open(sFileLabels, mode='r') as infile:
            reader = csv.reader(infile)
            dID = {rows[0]:rows[1] for rows in reader} #participant ID
        with open(sFileLabels, mode='r') as infile:
            reader = csv.reader(infile)
            dOnOff= {rows[0]:rows[2] for rows in reader} #on-off label
        with open(sFileLabels, mode='r') as infile:
            reader = csv.reader(infile)
            dDys={rows[0]:rows[3] for rows in reader} #dyskinesia label
        with open(sFileLabels, mode='r') as infile:
            reader = csv.reader(infile)
            dTrem={rows[0]:rows[4] for rows in reader} #tremor label

        #label selection
        if sTypeLabel=='on_off':
            dLabels=dOnOff
        elif sTypeLabel=='tremor':
            dLabels=dTrem
        elif sTypeLabel=='dyskinesia':
            dLabels=dDys
        else:
            print('sTypeLabel undefined')
        
    iNumFiles=len(lFilesPred)
    print('Number of analyzed file inputs: '+ str(iNumFiles))
    vPredIter=np.zeros((1,iNumFiles))
    mPredictions=np.zeros((1,iNumFiles))
    lID=[] # measurement_id 
    lDicts=[] 
    vLabels = [] # true label
    vParID=[] # participant ID

    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (acoustic, w-embed)
        lDicts.append(dPred)
   
    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            lID.append(k) # measurement_id 
            if not bTestSubset:
                vLabels.append(float(np.asarray(dLabels[k]))) #true labels
                vParID.append(float(np.asarray(dID[k]))) #participant ID
            vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            # Go through the second and next predictions files
            for j in range(1, iNumFiles):
                # fPred will contain the prediction of the predictions file we're going through
                fPred=lDicts[j].get(k)
                if fPred: # if we found a prediction
                    # we add it in the array
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    print(['Unkwnown key:' + k])
                    vPredIter[0,j]=float(np.asarray(lDicts[0][k]))
            if bEnter==1:
                # if it's the first prediction file, we initialize it
                mPredictions=vPredIter.copy() # mPredictions initialization.
                bEnter=0
            else:
                # We add the new predictions as a new column if it's not the first file we're going through
                mPredictions=np.append(mPredictions,vPredIter,axis=0)

    vAverage=np.mean(mPredictions,axis=1)
    if bRound==1:
        vPrediction=np.round(vAverage)
    else:
        vPrediction=vAverage
    
    vRes1=mPredictions[:,[0]]
    vRes2=mPredictions[:,[1]]

    lID, vPrediction=zip(*sorted(zip(lID, vPrediction)))
    df = pd.DataFrame({'measurement_id': lID, sTypeLabel:vPrediction})
    print('Submission file was created: '+sDirOut+"submissionCisPD"+sTypeLabel+".csv")
    df.to_csv(sDirOut+'submissionCisPD'+sTypeLabel+'.csv', index=False)
    
    return vRes1, vRes2, vAverage, vParID, vLabels

### Dyskinesia

### Get predictions file on test subset 

In [ ]:
# tsfresh predictions file on test kfolds 
sFilePred1='<path-to-github-repo>/tsfresh/submit/submission/cis-pd.dyskinesia.perpatient.csv'

# SVR predictions files on test kfolds 
sFilePred2='<path-to-kaldi>/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred2]

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dysk_test"

sTypeLabel = "dyskinesia"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0

# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel, True)


### Example on test folds

In [ ]:
# tsfresh predictions file on test kfolds 
sFilePred1='<path-github-repo>/tsfresh/submit/submission/cis-pd.dyskinesia.perpatient.csv'

# SVR predictions files on test kfolds 
sFilePred2='<path-to-kaldi>/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_650/resiVecPerPatientSVR_Fold_all_goodparams/preds_per_patient.csv'

lFilesPred=[sFilePred1,sFilePred2]

# Path to labels on CIS-PD 
sFileLabels='<path-to-data>/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

# Folder where we want to save the csv file with the average results 
dest_dir=sDirOut 

# Name of the CSV file to be created with the average 
fileName ="dysk_test"

sTypeLabel = "dyskinesia"

# Flag to round to the nearest integer or not 
# BUG, FIXME: It doesn't seem like this changes anything? 
bRound=0


# Fusion with average of predictions get_final_score(vPredictions, vParID, vTrueLabels)
vRes1, vRes2, vAverage, vParID, vLabels = average_fusion(lFilesPred, dest_dir, fileName, bRound, sFileLabels, sTypeLabel)

print('Overall MSE Classif. 1 - tsfresh: ', get_final_score(vRes1, np.array(vParID).astype(int), vLabels))
print('Overall MSE Classif. 2 - ivec: ', get_final_score(vRes2, vParID, vLabels))
print('Overall MSE Fusion - average : ', get_final_score(vAverage, vParID, vLabels))


